In [ ]:
import torch
import torchvision
import torchvision.transforms as T
from matplotlib import pyplot as plt
import numpy as np
transform=T.Compose([T.ToTensor(), T.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

traindata=torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
trainloader=torch.utils.data.DataLoader(traindata, batch_size=4, shuffle=True)

testdata=torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
testloader=torch.utils.data.DataLoader(testdata, batch_size=4, shuffle=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def imgshow(image):
    image=image*0.5 + 0.5
    img_np = image.numpy()
    plt.imsave('sample.jpg', np.transpose(img_np,(1,2,0)))

image,labels=iter(trainloader).next()
imgshow(torchvision.utils.make_grid(image))
print( [classes[labels[j]] for j in range(4)])

import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.cnv1=nn.Conv2d(3,6,5)
        self.pool=nn.MaxPool2d(2,2)
        self.cnv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(400,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)

    def forward(self,x):
        x = self.pool(F.relu(self.cnv1(x)))
        x = self.pool(F.relu(self.cnv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        

net=Network()
import torch.optim as optim
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters() , lr=0.001, momentum=0.9)



In [ ]:
from matplotlib import pyplot as plt
def imageshow(im):
    im_np= im.numpy()
    im_np= np.transpose(im_np, (1,2,0))
    im_np=im_np*0.5 + 0.5 
    plt.imshow(im_np)
images,labels = iter(trainloader).next()
images=torchvision.utils.make_grid(images)
imageshow(images)
#images

In [ ]:
for epoch in range(2):
    running_loss=0
    for i,data in enumerate(trainloader):
        images,labels=data
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        

In [ ]:
from sklearn.metrics import recall_score
correct=0
recall=[]
total=0
classes_ref=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
classes_total=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
with torch.no_grad():
    for data in testloader:
        image,labels=data
        y_true = labels.view(-1).numpy()
        output_test=net(image)
        _, predictions = torch.max(output_test,1)
        total+=labels.size(0)
        y_pred = predictions.view(-1).numpy()
        correct+=(predictions==labels).sum().item()
        recall.append(recall_score(list(y_true), list(y_pred), average='macro'))
        for i in range(4):
            if predictions[i]==labels[i]:
                classes_ref[np.int(predictions[i].numpy())]+=1
            classes_total[labels[i]]+=1

    print('Accuracy={}%'.format(100*correct/total))
    print('Recall ={} '.format(np.mean(recall)))






In [ ]:
for i in range (10):
    print('Accuracy of {} = {}'.format(classes[i],classes_ref[i]/10))

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import torch
import torchvision
from torchvision import datasets,models,transforms
import numpy as np
from matplotlib import pyplot as plt
import os
from skimage import io
import pathlib
from torch.autograd import Variable

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from sklearn.metrics import recall_score

"""
import argparse
parser = argparse.ArgumentParser()

parser.add_argument('-l','--filelist', nargs='+', help='<Required> Set flag', required=True)

args = parser.parse_args()
"""

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy


ngpu=4

root='/ssd_scratch/cvit/ashish/kidney/'

T = {'train': transforms.Compose([ transforms.RandomResizedCrop(224),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.596, 0.436, 0.586], [0.2066, 0.240, 0.186])       
                                 ]) ,
      
    'val': transforms.Compose([   transforms.Resize(224),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.596, 0.436, 0.586], [0.2066, 0.240, 0.186])       
                                 ]) ,
      
    'test': transforms.Compose([ transforms.Resize(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.596, 0.436, 0.586], [0.2066, 0.240, 0.186])       
                                 ]) 
    }

datasets={x: datasets.ImageFolder( os.path.join(root,x), T[x] ) for x in ['test'] }
dataloaders={x : torch.utils.data.DataLoader(datasets[x] , shuffle=True, batch_size=128 , num_workers=4) for x in ['test']}
datalength= { x : len(datasets[x]) for x in ['test']}
print(datalength['test'])
device=torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
class_names=datasets['test'].classes
print(class_names)



In [2]:
t=[1,0,0,0,0,0,0,0,0,0,0 ]
if 1 in t:
    print('cancer')
    
    
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import torch
import torchvision
from torchvision import datasets,models,transforms
import numpy as np
from matplotlib import pyplot as plt
import os
from skimage import io
import pathlib


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

"""
import argparse
parser = argparse.ArgumentParser()

parser.add_argument('-l','--filelist', nargs='+', help='<Required> Set flag', required=True)

args = parser.parse_args()
"""

import argparse
parser = argparse.ArgumentParser()

parser.add_argument('--load')
parser.add_argument('--savemodel')
parser.add_argument('--dataroot')
parser.add_argument('--ngpu')

args = parser.parse_args()


g=args.load
print(g)
fg=pathlib.Path(g)

s=args.savemodel
print(s)



import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy


ngpu=args.ngpu

root = args.dataroot

T = {'train': transforms.Compose([ transforms.RandomResizedCrop(224),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.596, 0.436, 0.586], [0.2066, 0.240, 0.186])       
                                 ]) ,
      
    'valid': transforms.Compose([   transforms.Resize(224),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.596, 0.436, 0.586], [0.2066, 0.240, 0.186])       
                                 ]) ,
      
    'test': transforms.Compose([ transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))       
                                 ]) 
    }

datasets={x: datasets.ImageFolder( os.path.join(root,x), T[x] ) for x in os.listdir(root) }
dataloaders={x : torch.utils.data.DataLoader(datasets[x] , shuffle=True, batch_size=128 , num_workers=4) for x in os.listdir(root)}
datalength= { x : len(datasets[x]) for x in os.listdir(root)}
print(datalength['train'])
print(datalength['valid'])
device=torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
class_names=datasets['train'].classes
print(class_names)


"""
def imgshow(img):
    im=img.numpy().transpose((1,2,0))
    mean = np.array([0.596, 0.436, 0.586])
    std = np.array([0.2066, 0.240, 0.186])
    im=im*std + mean
    plt.imsave('images.jpg',im)




images,labels= iter(dataloaders['train']).next()
images=torchvision.utils.make_grid(images)
imgshow(images)
"""
lst=os.listdir(root)
acc_list=[]
loss_list=[]
epoch_list=[]
#lst=lst.sort()
print(lst)
def train_model(model, criterion, optimizer, scheduler , num_epochs=10):
    since=time.time()
    best_model_wts=copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('-'*10 + ' Epoch {}'.format(epoch) + '-'*10, flush=True)
            
        for phase in ['train', 'valid']:
            print(phase,flush=True)
            if phase=='train':
                model.train()
            else:
                model.eval()
            running_loss=0
            running_corrects=0
            try:
                for images , labels in dataloaders[phase]:
                    images=images.to(device)
                    labels=labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase=='train'):

                        outputs=model(images)
                        _,predictions=torch.max(outputs,1)
                        loss=criterion(outputs,labels)
                        if phase=='train':
                            loss.backward()
                            optimizer.step()
                    running_loss+=loss.item()*images.size(0)
                    running_corrects+=torch.sum(predictions == labels.data)
                epoch_loss=running_loss/datalength[phase]
                epoch_acc=running_corrects.double()/datalength[phase]
                if phase=='train':
                	scheduler.step(epoch_loss)
                acc_list.append(epoch_acc)
                loss_list.append(epoch_loss)
                epoch_list.append(epoch)
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc),flush=True)


                if phase == 'valid' and epoch_acc>best_acc:
                    best_acc=epoch_acc
                    best_model_wts=copy.deepcopy(model.state_dict())
            except Exception as e:
               print('exception occured',e,flush=True)
    model.load_state_dict(best_model_wts)    
    return model




model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)


"""
d='/home/ashishmenon/openslide_WSI/synth_pretraining/pretrained_KIRP.pth'

fd=pathlib.Path(d)
"""





if (fg.exists ()):
    print('Loading Model', flush=True)
    model_ft = model_ft.to(device)
    if (device.type == 'cuda') and (ngpu > 1):
        model_ft = nn.DataParallel(model_ft, list(range(ngpu)))
    model_ft.load_state_dict(torch.load(d))
    print('Model Loaded', flush=True)


if (not(fg.exists ())):

    model_ft = model_ft.to(device)
    if (device.type == 'cuda') and (ngpu > 1):
    model_ft = nn.DataParallel(model_ft, list(range(ngpu)))

criterion = nn.CrossEntropyLoss()


optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.00005, weight_decay=0.05)
exp_lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, 'min', patience=2, verbose=True, factor = 0.2)



model_conv = train_model(model_ft, criterion, optimizer_ft,exp_lr_scheduler, num_epochs=25)
torch.save(model_conv.state_dict(),s)

plt.plot(epoch_list,acc_list)
plt.savefig('acc.jpg')


plt.plot(epoch_list,loss_list)
plt.savefig('loss.jpg')

cancer
